In [1]:
# Import live code changes in
%load_ext autoreload
%autoreload 

from pathlib import Path
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import qmc
from sklearn.preprocessing import MinMaxScaler

from sovereign.flood import build_basin_curves, BasinLossCurve, risk_data_future_shift, run_simulation, extract_sectoral_losses
from sovereign.macroeconomic import run_flood_sim_for_macro, prepare_DIGNAD, run_DIGNAD

In [2]:
root = Path.cwd().parent # find project root
THA_calibration_path = os.path.join(root, "inputs", "macro", "THA_2022_calibration_final.csv")
risk_basin_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins', 'risk_basins.csv')
copula_path = os.path.join(root, 'outputs', 'flood', 'dependence', 'copulas')
risk_data = pd.read_csv(risk_basin_path)
future_rp_shifts = pd.read_csv(os.path.join(root, 'outputs', 'flood', 'future', 'basin_rp_shifts.csv'))
# Drop first "unnamed column"
risk_data = risk_data.iloc[:, 1:]
# Add AEP column
risk_data['AEP'] = 1 / risk_data['RP']
# Add a column converting current prorection level into AEP
risk_data['Pr_L_AEP'] = np.where(risk_data['Pr_L'] == 0, 0, 1 / risk_data['Pr_L']) # using numpy where avoids zero division errors
risk_data.reset_index(drop=True, inplace=True)

In [3]:
#### 1. Flood simulation parameters
adaptation_aep = 0.01 # 100-year flood protection
n_years = 10000 # number of years to simulate
Thai_GDP = 496e9 # 2022 numbers in USD
future_hydro = 'jules-w2'
future_epoch = 2070
future_scenario = 'ssp585'
future_stat = 'q90'
# National GVA figures from DOSE
agr_GVA = 42880325598
man_GVA = 162659433017
ser_GVA = 316647741231
# Disaggregate output losses
TRADABLE_SHARES = {
    "Agriculture": 1.0,
    "Manufacturing": 0.7,
    "Service": 0.5,
}
#### 2. Macroeconomic model parameters
sim_start_year = 2022
nat_disaster_year = 2027
recovery_period = 3 # years
adaptation_cost = 21.97 # billion
reconstruction_efficiency = 0 # non-adjustable parameter
public_debt_premium = 0 # non-adjustable parameter
gdp_avg_years = 5 # we are intereseted in calculating average GDP impact over this period

In [4]:
# Adjust future risk data
future_risk_data = risk_data_future_shift(risk_data, future_rp_shifts, future_hydro, future_scenario, future_epoch, future_stat, degrade_protection=True)

In [5]:
# Build basin loss probability curves
baseline_curves: dict[int, BasinLossCurve] = build_basin_curves(risk_data)
future_curves: dict[int, BasinLossCurve] = build_basin_curves(future_risk_data)

In [6]:
# Load copula data
copula_random_numbers = pd.read_parquet(os.path.join(copula_path, "copula_random_numbers.gzip"))

In [7]:
baseline_current, baseline_adapted = run_flood_sim_for_macro(
    baseline_curves, adaptation_aep, n_years, copula_random_numbers, agr_GVA, man_GVA, ser_GVA, TRADABLE_SHARES, Thai_GDP
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:38<00:00, 258.43it/s]


In [8]:
future_current, future_adapted = run_flood_sim_for_macro(
    future_curves, adaptation_aep, n_years, copula_random_numbers, agr_GVA, man_GVA, ser_GVA, TRADABLE_SHARES, Thai_GDP
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:45<00:00, 221.04it/s]


In [9]:
df_combined = pd.concat([baseline_current, future_current], ignore_index=True)

In [10]:
param_ranges = {
    'dY_T': (df_combined['dY_T'].min(), 
                       df_combined['dY_T'].max()),
    'dY_N': (df_combined['dY_N'].min(),
                          df_combined['dY_N'].max()),
    'dK_priv': (df_combined['dK_priv'].min(),
                     df_combined['dK_priv'].max()),
    'dK_pub': (df_combined['dK_pub'].min(),
                      df_combined['dK_pub'].max())
}

In [11]:
def create_dignad_parameter_grid(param_ranges, n_samples=500):
    """
    Create efficient parameter grid using Latin Hypercube Sampling
    """
    # Define parameter bounds
    n_params = len(param_ranges)
    sampler = qmc.LatinHypercube(d=n_params)
    sample = sampler.random(n=n_samples)
    
    # Scale to actual ranges
    param_names = list(param_ranges.keys())
    scaled_samples = []
    
    for i, param in enumerate(param_names):
        min_val, max_val = param_ranges[param]
        scaled_values = sample[:, i] * (max_val - min_val) + min_val
        scaled_samples.append(scaled_values)
    
    # Create DataFrame
    param_grid = pd.DataFrame({
        param: values for param, values in zip(param_names, scaled_samples)
    })
    
    # Add fixed parameters if needed
    param_grid['share_tradable'] = 0.5
    param_grid['reconstruction_efficiency'] = 0
    param_grid['public_debt_premium'] = 0
    
    return param_grid
param_grid = create_dignad_parameter_grid(param_ranges, n_samples=100)

In [12]:
def precompute_dignad_surface(param_grid,
                              save_path='dignad_response_surface.pkl',
                              chunk_size=4,
                              resume=True):
    """
    Run DIGNAD for all parameter combinations and save results
    with periodic checkpointing and optional resume.
    """
    save_path = Path(save_path)

    results = []
    done_idxs = set()

    # === Try to resume from existing file ===
    if resume and save_path.exists():
        prev = pd.read_pickle(save_path)
        results = prev.to_dict(orient="records")

        if "row_idx" in prev.columns:
            done_idxs = set(prev["row_idx"])
            print(f"Resuming: {len(done_idxs)} parameter sets already done.")
        else:
            # fallback: assume index matches param_grid.index
            done_idxs = set(prev.index)
            print(f"Resuming (no row_idx column): {len(done_idxs)} done.")

    # === Main loop ===
    for loop_i, (idx, row) in enumerate(tqdm(param_grid.iterrows(),
                                             total=len(param_grid))):
        if idx in done_idxs:
            continue  # skip ones we already ran

        # ---- Run DIGNAD ----
        gdp_impact, years = run_DIGNAD(
            sim_start_year, nat_disaster_year, recovery_period,
            row['dY_T'], row['dY_N'],
            row['reconstruction_efficiency'], row['public_debt_premium'],
            row['dK_pub'], row['dK_priv'],
            row['share_tradable']
        )

        # ---- Store results ----
        result_entry = row.to_dict()
        result_entry['row_idx'] = idx  # so we can resume reliably
        t_shock = nat_disaster_year - sim_start_year
        result_entry['gdp_series'] = gdp_impact
        result_entry['gdp_avg'] = np.mean(
            gdp_impact[t_shock : t_shock + gdp_avg_years]
        )
        results.append(result_entry)

        # === checkpoint every chunk_size NEW runs ===
        if (len(done_idxs) + sum(r['row_idx'] not in done_idxs
                                 for r in results)) % chunk_size == 0:
            df_checkpoint = pd.DataFrame(results)
            df_checkpoint.to_pickle(save_path)
            print(f"Checkpoint saved: {len(df_checkpoint)} rows.")

    # === Final save ===
    results_df = pd.DataFrame(results)
    results_df.to_pickle(save_path)
    print(f"Finished. Total rows: {len(results_df)}")

    return results_df


In [13]:
# Calibrate DIGNAD
prepare_DIGNAD(THA_calibration_path, adaptation_cost=adaptation_cost)

In [14]:
file_path=r"C:\Users\smit0210\OneDrive - Nexus365\Sovereign-Risk\Analysis\DIGNAD\pre_sim\pre_sim_n100_adapt.csv"
# Run precompution of DIGNAD
precompute_dignad_surface(param_grid, save_path=file_path, chunk_size=20)

  8%|██████▍                                                                         | 8/100 [08:37<1:39:12, 64.70s/it]

MATLAB script not executed succesfully


TypeError: 'NoneType' object is not subscriptable

In [15]:
param_grid

,dY_T,dY_N,dK_priv,dK_pub,share_tradable,reconstruction_efficiency,public_debt_premium
0,0.130919,0.127458,0.015618,0.045507,0.5,0,0
1,0.020304,0.210074,0.162419,0.012639,0.5,0,0
2,0.079255,0.100020,0.393821,0.153846,0.5,0,0
3,0.107464,0.268937,0.429892,0.079121,0.5,0,0
4,0.112826,0.068743,0.307466,0.051486,0.5,0,0
...,...,...,...,...,...,...,...
95,0.006019,0.196065,0.104926,0.056743,0.5,0,0
96,0.047571,0.207427,0.121647,0.060746,0.5,0,0
97,0.211848,0.228865,0.198129,0.019310,0.5,0,0
98,0.122402,0.006332,0.041401,0.155751,0.5,0,0


In [17]:
param_grid[:10]

,dY_T,dY_N,dK_priv,dK_pub,share_tradable,reconstruction_efficiency,public_debt_premium
0,0.130919,0.127458,0.015618,0.045507,0.5,0,0
1,0.020304,0.210074,0.162419,0.012639,0.5,0,0
2,0.079255,0.100020,0.393821,0.153846,0.5,0,0
3,0.107464,0.268937,0.429892,0.079121,0.5,0,0
4,0.112826,0.068743,0.307466,0.051486,0.5,0,0
5,0.184026,0.226274,0.317380,0.089282,0.5,0,0
6,0.085115,0.269720,0.234091,0.133686,0.5,0,0
7,0.044380,0.108339,0.410468,0.073912,0.5,0,0
8,0.145689,0.165301,0.109161,0.112580,0.5,0,0
9,0.114893,0.019746,0.348562,0.027385,0.5,0,0
